In [1]:
import numpy as np
import pandas as pd
import json

In [2]:
def loadJson(path):
    '''returns dataframe from json file'''
    with open(path,'r') as fp: 
        tweets_json = json.load(fp)
    return pd.read_json(tweets_json)

In [16]:
def saveDataFrame(df,path):
    '''save DataFerame in csv format'''
    df.to_csv(path,sep=',',index=False)

In [4]:
def getData(l):
    user=[]
    ht=[]
    urls=[]
    ment=[]
    if not(type(l)==list):
        return 'Null,Null,Null,Null'
    for t in l:
        user = t['user']['name']        
        if bool (t['entities']['hashtags']) :  # if not empty
            #get hashtags
            for h in (t['entities']['hashtags']):
                ht.append(h['text'])
        if bool (t['entities']['urls']) :  # if not empty
            #get urls
            for u in (t['entities']['urls']):
                urls.append(u['display_url'])
        if bool (t['entities']['user_mentions']) :  # if not empty
            #get mentions
            for m in (t['entities']['user_mentions']):
                ment.append(m['name'])
    c = [user, ht, urls, ment]
    return (c)

In [5]:
def generateData(json_df):
    df = pd.DataFrame() #aux DataFrame
    date = json_df.columns[0]
    df['data'] = json_df[date].apply(getData) #Collect all attributes into a single column
    df['User'] = df['data'].apply(lambda x: x[0])
    df['Hashtags'] = df['data'].apply(lambda x: x[1])
    df['Links'] = df['data'].apply(lambda x: x[2])
    df['Mentions'] = df['data'].apply(lambda x: x[3])
    df= df.drop('data', axis=1)
    return df

In [6]:
def generateHTs(data):    
    hashtags=pd.DataFrame()
    hashtags['User']=np.NaN
    hashtags['Hashtags']=np.NaN
    for i in (data.index):
        hts = data.loc[i]['Hashtags'] 
        user = [data.loc[i]['User']]*len(hts)
        df = pd.DataFrame()
        df['User']=user
        df['Hashtags']=hts
        hashtags=hashtags.append(df,ignore_index=True)
    return (hashtags)

In [7]:
def generateLinks(data):    
    links=pd.DataFrame()
    links['User']=np.NaN
    links['Links']=np.NaN
    for i in (data.index):
        lnks = data.loc[i]['Links'] 
        user = [data.loc[i]['User']]*len(lnks)
        df = pd.DataFrame()
        df['User']=user
        df['Links']=lnks
        links=links.append(df,ignore_index=True)
    return (links)

In [8]:
def generateMentions(data):    
    mentions=pd.DataFrame()
    mentions['User']=np.NaN
    mentions['Mentions']=np.NaN
    for i in (data.index):
        m = data.loc[i]['Mentions'] 
        user = [data.loc[i]['User']]*len(m)
        df = pd.DataFrame()
        df['User']=user
        df['Mentions']=m
        mentions=mentions.append(df,ignore_index=True)
    return (mentions)

In [14]:
def generateGraphDB(json_df):
    #Check this empty save_to, haven't tried it
    save_to=''
    data=generateData(json_df)
    hts=generateHTs(data)
    links=generateLinks(data)
    mentions=generateMentions(data)
    saveDataFrame(hts,path=save_to+'Hashtags.csv')
    saveDataFrame(links,path=save_to+'Links.csv')
    saveDataFrame(mentions,path=save_to+'Mentions.csv')  

In [10]:
file = loadJson('C:/Users/Raquel/Downloads/todaysTweets_2018-03-01.json')

In [17]:
generateGraphDB(file)

In [9]:
l1= file['2018-03-01'][80]